**Don't change the grader**. It will be used to grade your code as is. Fix any errors in your library.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Your Code Imported Here

In [2]:
from csc665 import features, metrics
import csc665.ensemble

/Users/kalininalex/miniconda3/envs/py35_intel/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Prepare

In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import unittest as ut
import sklearn.ensemble

class Object(object): pass
class TC(ut.TestCase): pass
tc = TC()

In [4]:
def compare_models(sk_model, our_model, X_train, X_test, y_train, y_test):
    sk_model.fit(X_train, y_train)
    our_model.fit(X_train, y_train)
#     print(our_tree)
    
    expected = metrics.rsq(sk_model.predict(X_train), y_train)
    actual = metrics.rsq(our_model.predict(X_train), y_train)
    print("Train, sklearn, ours:", expected, actual)
    tc.assertAlmostEqual(expected, actual)
    
    expected = metrics.rsq(sk_model.predict(X_test), y_test)
    actual = metrics.rsq(our_model.predict(X_test), y_test)
    print("Test, sklearn, ours:", expected, actual)
    tc.assertLess(abs(expected - actual), 0.19)

# Income Dataset

In [5]:
income_df = pd.read_csv("http://www-bcf.usc.edu/~gareth/ISL/Income2.csv")
income_df = income_df.drop("Unnamed: 0", axis=1)
_income = features.Object()
_income.X = income_df.drop("Income", axis=1)
_income.y = income_df["Income"].values

In [6]:
import sklearn.model_selection
RAND_STATE = 17

_income.X_train, _income.X_test, _income.y_train, _income.y_test = \
    sklearn.model_selection.train_test_split(
        _income.X, 
        _income.y, 
        test_size=0.2, 
        random_state=RAND_STATE)

### Income Dataset

In [11]:
compare_models(
    sklearn.ensemble.RandomForestRegressor(n_estimators=1, random_state=RAND_STATE),
    csc665.ensemble.RandomForestRegressor(n_estimators=1, sample_ratio=1.0, random_state=RAND_STATE),
    _income.X_train, _income.X_test, _income.y_train, _income.y_test)

Train, sklearn, ours: 0.9364717923402179 0.9364717923402179
Test, sklearn, ours: 0.721713329027837 0.721713329027837


In [12]:
compare_models(
    sklearn.ensemble.RandomForestRegressor(n_estimators=10, random_state=RAND_STATE),
    csc665.ensemble.RandomForestRegressor(n_estimators=10, sample_ratio=1.0, random_state=RAND_STATE),
    _income.X_train, _income.X_test, _income.y_train, _income.y_test)

Train, sklearn, ours: 0.9843304440755631 0.9843304440755631
Test, sklearn, ours: 0.8274958775158834 0.8274958775158834


In [13]:
compare_models(
    sklearn.ensemble.RandomForestRegressor(n_estimators=100, random_state=RAND_STATE),
    csc665.ensemble.RandomForestRegressor(n_estimators=100, sample_ratio=1.0, random_state=RAND_STATE),
    _income.X_train, _income.X_test, _income.y_train, _income.y_test)

Train, sklearn, ours: 0.9889569097665746 0.9889569097665746
Test, sklearn, ours: 0.883233461265201 0.883233461265201


# Melbourne Dataset - Sampled

We'll use only 200 datapoints from Melbourne dataset for performance reasons.

In [18]:
csv_df = pd.read_csv("data/Melbourne_housing_FULL.csv")

In [22]:
_mel = Object()
_mel.X, _mel.y = features.preprocess_ver_1(csv_df, "Price")

In [23]:
print(_mel.X.shape, _mel.y.shape)

assert _mel.X.shape[0] == 8887, "X.shape"
assert _mel.y.shape[0] == 8887, "y.shape"

(8887, 20) (8887,)


In [27]:
_mel_sampled = Object()
_mel_sampled.X_train = _mel.X[:200]
_mel_sampled.y_train = _mel.y[:200]
_mel_sampled.X_test = _mel.X[:50]
_mel_sampled.y_test = _mel.y[:50]

In [28]:
compare_models(
    sklearn.ensemble.RandomForestRegressor(n_estimators=1, random_state=RAND_STATE),
    csc665.ensemble.RandomForestRegressor(n_estimators=1, sample_ratio=1.0, random_state=RAND_STATE),
    _mel_sampled.X_train, _mel_sampled.X_test, _mel_sampled.y_train, _mel_sampled.y_test)

Train, sklearn, ours: 0.920353425859184 0.920353425859184
Test, sklearn, ours: 0.8391223876039476 0.8391223876039476
